In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!unzip /content/drive/MyDrive/red_data_no_webp.zip



Streaming output truncated to the last 5000 lines.
  inflating: red_data/img/Metal/metal_02_ (312).jpg  
  inflating: red_data/img/Metal/metal_02_ (313).jpg  
  inflating: red_data/img/Metal/metal_02_ (314).jpg  
  inflating: red_data/img/Metal/metal_02_ (315).jpg  
  inflating: red_data/img/Metal/metal_02_ (316).jpg  
  inflating: red_data/img/Metal/metal_02_ (317).jpg  
  inflating: red_data/img/Metal/metal_02_ (318).jpg  
  inflating: red_data/img/Metal/metal_02_ (319).jpg  
  inflating: red_data/img/Metal/metal_02_ (32).jpg  
  inflating: red_data/img/Metal/metal_02_ (320).jpg  
  inflating: red_data/img/Metal/metal_02_ (321).jpg  
  inflating: red_data/img/Metal/metal_02_ (322).jpg  
  inflating: red_data/img/Metal/metal_02_ (323).jpg  
  inflating: red_data/img/Metal/metal_02_ (324).jpg  
  inflating: red_data/img/Metal/metal_02_ (325).jpg  
  inflating: red_data/img/Metal/metal_02_ (326).jpg  
  inflating: red_data/img/Metal/metal_02_ (327).jpg  
  inflating: red_data/img/Metal/

In [ ]:
import os

def list_file_extensions(directory):
    # Store unique file extensions
    extensions = set()

    # Walk through the directory
    for root, dirs, files in os.walk(directory):
        for file in files:
            # Get the file extension and add to the set (ignoring empty extensions)
            ext = os.path.splitext(file)[1]
            if ext:  # Only add extensions that are not empty
                extensions.add(ext.lower())

    # Return sorted list of unique extensions
    return sorted(extensions)

# Example usage:
directory_path = input("Enter the directory path: ")
extensions = list_file_extensions('/content/red_data/img/')

print("Unique file extensions in the directory:")
for ext in extensions:
    print(ext)


Enter the directory path: fvgbnm,.
Unique file extensions in the directory:
.jpeg
.jpg


In [ ]:
import os

extracted_files = os.listdir('/content/red_data/img/')
print(extracted_files)


['Metal', 'Plastic-Regular', 'Glass', 'Paperboard', 'Plastic-Polystyrene']


Uses a transfer learning approach with ResNet50

In [ ]:
import tensorflow as tf
import numpy as np
import os
import pathlib
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from sklearn.utils import class_weight

# Set image dimensions
img_height = 240
img_width = 240
batch_size = 8

# Path to your image directory
str_path = '/content/red_data/img/'
data_dir = pathlib.Path(str_path)

# Load the datasets without applying cache/prefetch initially
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.15,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2),
])



val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.15,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

# Now access class_names before any dataset transformation
class_names = train_ds.class_names
print(class_names)  # You should be able to access class names here

train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y))

# Apply caching and prefetching after accessing class names
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# The rest of your code continues...

# Compute class weights (REMOVED BECAUSE IT TAKES TOO MUCH TIME)
#def get_labels(train_ds):
    #labels = []
    #for _, label in train_ds:
        #labels.extend(label.numpy())
    #return np.array(labels).astype(int)

#train_labels = get_labels(train_ds)
#class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(train_labels), y=train_labels)
#class_weights_dict = dict(enumerate(class_weights))
num_classes = 5
for images, labels in train_ds.take(1):
    print("Image shape:", images.shape)
# Define the base pre-trained model (ResNet50)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
base_model.summary()
# Freeze the base model (all layers initially frozen)
base_model.trainable = False

# Create the custom model on top of the pre-trained model
model = models.Sequential([
    base_model,  # Pre-trained ResNet50 model
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')  # Output layer with softmax
])

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

from sklearn.metrics import classification_report

def custom_metric(y_true, y_pred):
    return classification_report(y_true, y_pred, target_names=class_names)

# Callbacks
callbacks = [
    tf.keras.callbacks.ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_loss'),
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001)

]

# Train the model with class weights
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15,
    callbacks=callbacks,
    #class_weight=class_weights_dict
)



# After training the new layers, unfreeze some layers of the base model to fine-tune
base_model.trainable = True

# Freeze the earlier layers of ResNet50 to prevent them from being modified too much during fine-tuning
for layer in base_model.layers[:100]:  # You can experiment with which layers to unfreeze
    layer.trainable = False

from sklearn.metrics import classification_report

def custom_metric(y_true, y_pred):
    return classification_report(y_true, y_pred, target_names=class_names)

# Re-compile the model for fine-tuning with a lower learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Fine-tune the model
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15,
    callbacks=callbacks,
    #class_weight=class_weights_dict
)

# Save the fine-tuned model
model.save('recyclebot.keras')
print("Transfer learning completed and model saved.")


Found 7340 files belonging to 5 classes.
Using 6239 files for training.
Found 7340 files belonging to 5 classes.
Using 1101 files for validation.
['Glass', 'Metal', 'Paperboard', 'Plastic-Polystyrene', 'Plastic-Regular']
Image shape: (8, 240, 240, 3)
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


Model: "resnet50"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, 240, 240, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, 246, 246, 3)    │              0 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 120, 120, 64)   │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 120, 120, 64)   │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 120, 120, 64)   │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, 122, 122, 64)   │              0 │ conv1_relu[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, 60, 60, 64)     │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 60, 60, 64)     │          4,160 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 60, 60, 64)     │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 60, 60, 64)     │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 60, 60, 64)     │         36,928 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, 60, 60, 64)     │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, 60, 60, 64)     │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv       │ (None, 60, 60, 256)    │         16,640 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_3_conv       │ (None, 60, 60, 256)    │         16,640 │ conv2_block1_2_relu[0… │
│ (Conv2D)             

 Total params: 23,587,712 (89.98 MB)

 Trainable params: 23,534,592 (89.78 MB)

 Non-trainable params: 53,120 (207.50 KB)

Epoch 1/15
780/780 ━━━━━━━━━━━━━━━━━━━━ 113s 127ms/step - accuracy: 0.5252 - loss: 1.3940 - val_accuracy: 0.8583 - val_loss: 0.4073 - learning_rate: 1.0000e-04
Epoch 2/15
780/780 ━━━━━━━━━━━━━━━━━━━━ 13s 16ms/step - accuracy: 0.7565 - loss: 0.6686 - val_accuracy: 0.8756 - val_loss: 0.3596 - learning_rate: 1.0000e-04
Epoch 3/15
780/780 ━━━━━━━━━━━━━━━━━━━━ 13s 16ms/step - accuracy: 0.8163 - loss: 0.5200 - val_accuracy: 0.8919 - val_loss: 0.3298 - learning_rate: 1.0000e-04
Epoch 4/15
780/780 ━━━━━━━━━━━━━━━━━━━━ 13s 16ms/step - accuracy: 0.8437 - loss: 0.4344 - val_accuracy: 0.8883 - val_loss: 0.3279 - learning_rate: 1.0000e-04
Epoch 5/15
780/780 ━━━━━━━━━━━━━━━━━━━━ 13s 17ms/step - accuracy: 0.8709 - loss: 0.3794 - val_accuracy: 0.9173 - val_loss: 0.2822 - learning_rate: 1.0000e-04
Epoch 6/15
780/780 ━━━━━━━━━━━━━━━━━━━━ 12s 15ms/step - accuracy: 0.8855 - loss: 0.3165 - val_accuracy: 0.8983 - val_loss: 0.2978 - learning_rate: 1.0000e-04
Epoch 7/15
 69/780 ━━━━━━━━━━━━━━━━━━━━ 9s 13ms/st

KeyboardInterrupt: 